In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

loc = "angular_distr_el/CH9/ID_ALL_mol_e0_valid/EN_gate/MFPADs_multinew_std"

#low old binning (wrong?) (12,6)
# fileRCR = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCR = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCL = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")


#high new binning (24,12)
fileRCR = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileRCL = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileSCR = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileSCL = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

k=fileRCR[loc].keys()
d=dict(fileRCR[loc].classes()) # shows the type of each item
dict(fileRCR[loc]) # keys : values
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
# type(check)
# check._members()

In [ ]:
def getamesh(x,y,z,d):
    temp = pd.DataFrame(np.hstack((x[:,None], y[:,None], z[:,None])))
    temp.columns = ["x", "y", "z"]
    sph = PyntCloud(temp)
    x,  y,  z = sph.points["x"], sph.points["y"], sph.points["z"]
    pts = sph.points[['x', 'y', 'z']]
    delaun = structures.Delaunay3D(pts)
    
    delaun.compute()
    mesh = delaun.get_mesh()
    tri = mesh[['v1', 'v2', 'v3']].values
    I, J, K = tri.T
    return x,y,z,I,J,K

def makeamesh (x,y,z,d):
    points2d_trace=go.Scatter(x=x, y=y, mode='markers', marker_color='red', marker_size=6)
    point_trace=go.Scatter(x=x, y=y, 
                         mode='markers',
                         name='points',
                         marker_color='red',
                         marker_size=6)
    pts2d=np.array([x,y]).T
    tri=Delaunay(pts2d)
    delaunay_tri = tr.triangulation_edges(pts2d, tri.simplices, linewidth=1)

    i, j, k = tri.simplices.T
    my_mesh3d = go.Mesh3d(
                        x = x,
                        y = y,
                        z = z,
                        i=i, j=j, k=k,
                        colorscale='deep_r',
                        colorbar_thickness=25,
                        intensity=d, 
                        flatshading=True)
    
    points3d=np.array([x,y,z]).T
    delaun_tri3d=tr.triangulation_edges(points3d, tri.simplices)
    return delaunay_tri, point_trace, my_mesh3d, delaun_tri3d


def smoothgauss(MFPAD, sigmax, sigmay):
    """
    Fucntion the smooths with a 2D gaussian function 2D matrixes. For the experiments
    x = cos(theta), y = phi. Phi is twice ctheta and sigma should be the same.
    """
    # NOTE: the right order is y,x (from stackoverflow)
    sigma = [sigmay,sigmax] 
    Y = sp.ndimage.filters.gaussian_filter(MFPAD, sigma, mode="constant")
    return Y

def shift_func(a, flag=0.):
    """
    shifts a linearly monotally increasing array of the average of the first two elements, either along columns or rows.
    It is manly used to plot cosphi[:,0] in b1 maps. cosphi is an array of tuple [(ctheta,phi)].
    """
    #if it is a list = 72
    if a.shape[0]>12:
        c=a.reshape(12,6)
        flag=1.
        # if the first two elements of the first row are equal,
        # pick the first two elements along the first column
        # and computer the average. This number is the same for all elements of the array
        if c[0][0] == c[0][1]:
            b=(c[1][0]-c[0][0])/2
            a=np.add(c,b)
        else:
            b=(c[0][1]-c[0][0])/2
            a=np.add(c,b)
    else:
        if a[0][0] == a[0][1]:
            b=(a[1][0]-a[0][0])/2
            a=np.add(a,b)
        else:
            b=(a[0][1]-a[0][0])/2
            a=np.add(a,b)
    if flag == 1.:
        # a=np.ravel(a)
        # it returns an array with dimension (n,)
        a=a.reshape(-1)
    return a

def cosphi_func(key, cosphi):
    ctheta_nc = float((str(key).split("costheta_")[1]).split("_phi")[0])
    phi_nc = float((str(key).split("phi_")[1]).split(";")[0])
    # alternative method with import re
    #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1)
    cosphi.append((ctheta_nc, phi_nc))
    return cosphi

def import_MFPAD(file, MFPAD, cosphi, MFPAD_xy, ctheta, ctheta_c, ctheta_cred, run_MFPAD=0., run_cos=0.):
    """
    Loads the MFPADs and the cos(theta) from the .root files.
    6 inputs + a parameter
    NOTE: MFPAD_xy and ctheta_c have originally +1 dimensions compare to the z values.
    For the sake of iminiut, cos(theta) is centered on the middle of the bins.
    """
    for key, value in file[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            cosphi=cosphi_func(key,cosphi)
            temp=np.array(file[filename].numpy())
            MFPAD.append(temp[0]) # it is a list!
            if run_MFPAD == 0.:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_MFPAD=1.
        elif "cos(theta)" in filename.lower():
            temp=np.array(file[filename].numpy())
            ctheta.append(temp[0]) # it is a list!
            if run_cos == 0.:
                ctheta_c.append(temp[1])
                ctheta_cred.append(np.array((ctheta_c[0][1:] + ctheta_c[0][:-1])/2)) #! reduced of 1 dimension
                run_cos=1.
    return MFPAD, cosphi, MFPAD_xy, ctheta, ctheta_c, ctheta_cred

In [ ]:
# initilialises a list
MFPAD_RCR=[]; ctheta_RCR=[]; ctheta_xaxis_RCR=[]; cosphi_RCR=[];
MFPAD_RCL=[]; ctheta_RCL=[]; ctheta_xaxis_RCL=[]; cosphi_RCL=[];
MFPAD_SCR=[]; ctheta_SCR=[]; ctheta_xaxis_SCR=[]; cosphi_SCR=[];
MFPAD_SCL=[]; ctheta_SCL=[]; ctheta_xaxis_SCL=[]; cosphi_SCL=[];

MFPAD_xy=[];
ctheta_cred=[];

import_MFPAD(fileRCR, MFPAD_RCR, cosphi_RCR, MFPAD_xy, ctheta_RCR, ctheta_xaxis_RCR, ctheta_cred)
import_MFPAD(fileRCL, MFPAD_RCL, cosphi_RCL, MFPAD_xy, ctheta_RCL, ctheta_xaxis_RCL, ctheta_cred)
import_MFPAD(fileSCR, MFPAD_SCR, cosphi_SCR, MFPAD_xy, ctheta_SCR, ctheta_xaxis_SCR, ctheta_cred)
import_MFPAD(fileSCL, MFPAD_SCL, cosphi_SCL, MFPAD_xy, ctheta_SCL, ctheta_xaxis_SCL, ctheta_cred)

# converting into nparray
MFPAD_RCR=np.array(MFPAD_RCR); ctheta_RCR=np.array(ctheta_RCR); ctheta_xaxis_RCR=np.array(ctheta_xaxis_RCR[0]); cosphi_RCR=np.array(cosphi_RCR);
MFPAD_RCL=np.array(MFPAD_RCL); ctheta_RCL=np.array(ctheta_RCL); ctheta_xaxis_RCL=np.array(ctheta_xaxis_RCL[0]); cosphi_RCL=np.array(cosphi_RCL);
MFPAD_SCR=np.array(MFPAD_SCR); ctheta_SCR=np.array(ctheta_SCR); ctheta_xaxis_SCR=np.array(ctheta_xaxis_SCR[0]); cosphi_SCR=np.array(cosphi_SCR);
MFPAD_SCL=np.array(MFPAD_SCL); ctheta_SCL=np.array(ctheta_SCL); ctheta_xaxis_SCL=np.array(ctheta_xaxis_SCL[0]); cosphi_SCL=np.array(cosphi_SCL);

ctheta_cred=ctheta_cred[0];
cosphi=cosphi_RCR;

In [ ]:
#FOR LIGHT DIRECTION, 72 each
cosn=np.array([col[0] for col in cosphi]);
phin=np.array([col[1] for col in cosphi]);
#FOR LIGHT DIRECTION

#FOR MFPDAS from here onwards
#cosm (13,) phim (25,)
cosm=MFPAD_xy[0][1]
phim=MFPAD_xy[0][0]

#cosM (12,) phiM (24,)
cosM=(MFPAD_xy[0][1][1:] + MFPAD_xy[0][1][:-1])/2
phiM=(MFPAD_xy[0][0][1:] + MFPAD_xy[0][0][:-1])/2

#effective dimensions to avoid retyping
c=cosM.shape[0]
p=phiM.shape[0]

#cartesian product of unique values, (288,2) each
phicos_MFPAD = np.array(list(itertools.product(phiM, cosM)))
cosphi_MFPAD = np.array(list(itertools.product(cosM, phiM)))
# print(phicos_MFPAD,cosphi_MFPAD)

#(288,1) for each
ctheta_adm = np.array([col[0] for col in cosphi_MFPAD])
theta_rad = np.arccos(ctheta_adm)
phi_ang = np.array([col[1] for col in cosphi_MFPAD])
phi_rad = phi_ang * np.pi/180.

In [ ]:
#look to the values in b1 map and found the MFPAD. The phi - cos(theta) grid of light direction id indipendendt from the enantiomer
cosphi_sh=list(zip(shift_func(cosphi[:,0]),shift_func(cosphi[:,1])))
df=pd.DataFrame(cosphi_sh, columns=["ctheta","phi"])
df.loc[df["phi"] == -45]

In [ ]:
#insert hier the corrispondent index number coming from the comparison with the b1 map values
checkthis=27
MFPAD=MFPAD_SCR

In [ ]:
fig0, ax0 = plt.subplots(1,3, figsize=(20, 6.5),sharex='col')
fig0.subplots_adjust(hspace = .5, wspace=.5)

ax0[0].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),MFPAD[checkthis].T)
ax0[1].contourf(phiM,cosM,MFPAD[checkthis].T) #has to be transposed becasue how it has been defined in lmf2root
ax0[2].contourf(cosM,phiM,MFPAD[checkthis]) #this is the original x=ctheta, y=phi

fig0.tight_layout()

MFPAD[0].shape

In [ ]:
fig01,ax01 = plt.subplots(1,4, figsize=(21, 5))
ax01[0].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_RCL[checkthis],0.3,0.6).T)
ax01[0].set_title("MFPAD RCL")
ax01[1].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_RCR[checkthis],0.3,0.6).T)
ax01[1].set_title("MFPAD RCR")
ax01[2].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_SCL[checkthis],0.3,0.6).T)
ax01[2].set_title("MFPAD SCL")
ax01[3].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_SCR[checkthis],0.3,0.6).T)
ax01[3].set_title("MFPAD SCR")

In [ ]:
MFPAD_SCL_flip=[]
MFPAD_SCR_flip=[]

for i in range(72):
    temp1=np.flip(MFPAD_SCL[i], axis=0)
    temp2=np.flip(temp1, axis=1)
    MFPAD_SCL_flip.append(temp2)

MFPAD_SCL_flip=np.array(MFPAD_SCL_flip)


for i in range(72):
    temp1=np.flip(MFPAD_SCR[i], axis=0)
    temp2=np.flip(temp1, axis=1)
    MFPAD_SCR_flip.append(temp2)

MFPAD_SCR_flip=np.array(MFPAD_SCR_flip)

# temp2=np.flip(MFPAD_SCR, axis=0)
# MFPAD_SCR_flip=np.flip(temp2, axis=1)

In [ ]:
fig02,ax02 = plt.subplots(1,4, figsize=(21, 5))
ax02[0].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_RCL[checkthis],0.3,0.6).T)
ax02[0].set_title("MFPAD RCL")
ax02[1].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_RCR[checkthis],0.3,0.6).T)
ax02[1].set_title("MFPAD RCR")
ax02[2].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_SCL_flip[checkthis],0.3,0.6).T)
ax02[2].set_title("MFPAD SCL flipped")
ax02[3].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_SCR_flip[checkthis],0.3,0.6).T)
ax02[3].set_title("MFPAD SCR flipped")

In [ ]:
#from spherical to cartesian coordinates

counts=MFPAD[checkthis].reshape(-1)
x = counts * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(p,c)
y = counts * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(p,c)
z = counts * np.cos(theta_rad)
Z = z.reshape(p,c)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
#from spherical to cartesian coordinates
#SMOOTHED by sigmax (ctheta) and sigmay (phi)
# sigma phi (y) can be double the value of x

counts=smoothgauss(MFPAD[checkthis],0.3,0.6).reshape(-1)
xs = counts * np.sin(theta_rad) * np.cos(phi_rad)
Xs = xs.reshape(p,c)
ys = counts * np.sin(theta_rad) * np.sin(phi_rad)
Ys = ys.reshape(p,c)
zs = counts * np.cos(theta_rad)
Zs = zs.reshape(p,c)

ds = np.sqrt(xs**2+ys**2+zs**2)
d_matrixs = np.sqrt(Xs**2+Ys**2+Zs**2)

In [ ]:
fig1 = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                    marker=dict( size=5,color=d, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

fig1.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig1.show()

fig1s = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs, mode='markers',
                    marker=dict( size=20,color=ds, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

fig1s.update_layout(title="TFMeOx MFPADs EXP smoothed", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig1s.show()

In [ ]:
delaunay_tri, point_trace, my_mesh3d, delaun_tri3d = makeamesh(xs,ys,zs,ds)

fig2 = go.Figure(data=[delaunay_tri, point_trace])
fig2.update_layout(width=500, height=500);
# fig2.show()

my_layout = dict(width=800, height=600,
                 scene_camera_eye=dict(x=1.5, y=1.5, z=1), 
                 scene_aspectratio=dict(x=1.5, y=1.5, z=1))

fig2s = go.Figure(data=[my_mesh3d, delaun_tri3d])
# fig2s.show()

In [ ]:
xn,yn,zn,In,Jn,Kn = getamesh(x,y,z,d)

fig3 = go.Figure(go.Mesh3d(x=x, y=y, z=z,
                           alphahull=5,  
                        #    i=In, j=Jn, k=Kn, 
                           intensity=d,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig3.update_layout(width=800, height=800)
# fig1.show()


In [ ]:
fig4 = go.Figure(data=[go.Surface(z=Z, surfacecolor=d_matrix)])
fig4.update_layout(title='TFMeOx MFPADs EXP surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig4.show()

fig4s = go.Figure(data=[go.Surface(z=Zs, surfacecolor=d_matrixs)])
fig4s.update_layout(title='TFMeOx MFPADs EXP surf SMOOTHED', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig4s.show()


In [ ]:
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
import open3d as o3d
import trimesh

sys.path.append('..')
import open3d_tutorial as o3dtut

output_path=(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\OUTPUTS\\")

alpha_mesh=[]
poisson_mesh=[]
densities=[]

alpha_mesh = o3d.geometry.TriangleMesh()
pcd = o3d.geometry.PointCloud()
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals

camera_loc=np.array([0.,0.,-1.])


#load the point cloud
# SMOOTHED
point_cloud=np.array([xs,ys,zs]).T
#Standard
# point_cloud=np.array([x,y,z]).T
cloud = PyntCloud.from_instance("open3d", pcd)
pcd.points = o3d.utility.Vector3dVector(point_cloud)
#resise the scale of the sample
vox_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, 1.)

#presetn in all approaches of plc
kdtree = cloud.add_structure("kdtree")
testc = cloud.get_neighbors(k=5)
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)

# mesh.compute_vertex_normals()


#compute the normals
pcd.estimate_normals(); #mandatory
# pcd.normals = o3d.utility.Vector3dVector(point_cloud) #perfectly radial and long, independent by the tangent plane
# They see good for open surfaces, but not for my case
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05,max_nn=20));
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50));

#orient the normals
#Number of nearest neighbours: 5 is the minimum to have a closed surface with scale >= 2
pcd.orient_normals_consistent_tangent_plane(5)
# pcd.orient_normals_towards_camera_location(camera_loc)
# pcd.orient_normals_to_align_with_direction([0.,1.,0.])


#alpha-shape algorithm
# alpha=np.log10(0.5)
# alpha=0.1
# tetralpha_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)
# alpha_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha, tetralpha_mesh, pt_map) #it resturns zero for some reason


#Poisson algorithm
poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=1.5, linear_fit=False)
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)
# p_mesh_crop.compute_triangle_normals() # usually computed before rendering (??)


#Hull ball algoritm
radii = [0.08, 0.1, 0.2, 0.4]
radius = 1.5 * avg_dist
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))
# bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector(radii))


'''
CLEANING THE TRIANGLES
arget_number_of_triangles (int) – The number of triangles that the simplified mesh should have. It is not guaranteed that this number will be reached.
maximum_error (float, optional, default=inf) – The maximum error where a vertex is allowed to be merged
boundary_weight (float, optional, default=1.0) – A weight applied to edge vertices used to preserve boundaries
'''
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)
dec_mesh.remove_degenerate_triangles()
# dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
# dec_mesh.remove_non_manifold_edges()

#none of them is working with poisson
# p_mesh_crop =poisson_mesh.simplify_quadric_decimation(6000)
# p_mesh_crop.remove_unreferenced_vertices
# p_mesh_crop.remove_degenerate_triangles()
# p_mesh_crop.remove_duplicated_triangles()
# p_mesh_crop.remove_duplicated_vertices()
# p_mesh_crop.remove_non_manifold_edges()

#create the triangular mesh with the vertices and faces from open3d to be visualise with another software
tri_mesh = trimesh.Trimesh(np.asarray(dec_mesh.vertices), np.asarray(dec_mesh.triangles),vertex_normals=np.asarray(dec_mesh.vertex_normals))
tri_mesh_pois = trimesh.Trimesh(np.asarray(poisson_mesh.vertices), np.asarray(poisson_mesh.triangles),vertex_normals=np.asarray(poisson_mesh.vertex_normals))
# tri_mesh_pois_crop = trimesh.Trimesh(np.asarray(p_mesh_crop.vertices), np.asarray(p_mesh_crop.triangles),vertex_normals=np.asarray(p_mesh_crop.vertex_normals))
# trimesh.convex.is_convex(dec_mesh)

In [ ]:
type(tri_mesh_pois)
tri_mesh_pois.vertices

In [ ]:
# np.asarray(poisson_mesh.vertices).T
xgo, ygo, zgo =np.asarray(poisson_mesh.vertices).T

dgo = np.sqrt(xgo**2+ygo**2+zgo**2)

# asarray(p_mesh_crop.triangles)
# txgo, tygo, tzgo =np.asarray(p_mesh_crop.vertices).T[0]
# tdgo = np.sqrt(txgo**2+tygo**2+tzgo**2)

# np.asarray(poisson_mesh.vertex_normals)
# poisson_mesh.triangle_normals
tigo, tjgo, tkgo =np.asarray(poisson_mesh.triangles).T
ti, tj, tk = tri_mesh_pois.triangles.T

In [ ]:
#designing the surface colour
#densities are the real density of features
densities = np.asarray(densities)
density_colors = plt.get_cmap('viridis')((dgo - dgo.min()) / (dgo.max() - dgo.min()))
density_colors = density_colors[:, :3]

#works for the plotting in o3d
poisson_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)


o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", dec_mesh);
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", poisson_mesh);
# o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", p_mesh_crop);

# my_lods = lod_mesh_export(p_mesh_crop, [100000,50000,10000,1000,100], ".ply", output_path)
my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], ".ply", output_path)

# o3d.visualization.draw_geometries([pcd, p_mesh_crop], mesh_show_back_face=True)
# o3d.visualization.draw_geometries([pcd, poisson_mesh],mesh_show_back_face=True)
# o3d.visualization.draw_geometries([pcd, poisson_mesh[100000]],point_show_normal=True)

# tri_mesh_pois.show()

In [ ]:
fig5 = go.Figure(go.Mesh3d(x=xgo, y=ygo, z=zgo, 
                           # alphahull=3.5, 
                           i=tigo, j=tjgo, k=tkgo, 
                           # i=ti, j=tj, k=tk, 
                           intensity=dgo,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig5.update_layout(width=800, height=800)
fig5.show()


In [ ]:
# Go scatters do not provide a legend
fig6 = go.Figure(data=[go.Scatter3d(x=xgo, y=ygo, z=zgo,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=dgo,            # set color to an array/list of desired values
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig6.update_layout(title="TFMeOx MFPADs poisson scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig6.show()

# fig6 = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs,
fig6s = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        # color=ds,
                        color=d,
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig6s.update_layout(title="TFMeOx MFPADs smoothed scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))

fig6s.show()

In [ ]:
# tigo=np.asarray(poisson_mesh.vertex_normals).T[0]
np.asarray(poisson_mesh.vertex_normals)

In [ ]:
tri_mesh_pois.vertex_normals

In [ ]:
#SHORTCUTS from keyboard: n = show normals, q = quit, w = mesh
# o3d.visualization.draw_geometries([pcd, poisson_mesh],mesh_show_back_face=True)

In [ ]:
def rot3d(alpha,beta,gamma):  #planar rotation of alpha radians
    return np.array([[np.cos(alpha)*np.cos(beta), np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)], 
                      [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
                      [-np.sin(beta),              np.cos(beta)*np.sin(gamma),                                           np.cos(beta)*np.cos(gamma)]])

frames=[]

fig8 = go.Figure(go.Mesh3d(x=xgo, y=ygo, z=zgo,i=tigo, j=tjgo, k=tkgo, intensity=dgo,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05,facenormalsepsilon=0,vertexnormalsepsilon=0),lightposition=dict(x=100,y=100,z=1000)))

T = np.arange(0,  2*np.pi, 0.125)
xyz = np.stack((xgo, ygo, zgo))
for alpha in T:
    xr, yr, zr  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyz)#  batch rotation of all points (x,y,z) on the Mesh
    frames.append(go.Frame(data=[go.Mesh3d(x=xr, y=yr, z=zr)])) #z is the same in a 3d rotation about zaxis
fig8.update(frames=frames);    


fig8.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

fig8.update_layout(updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=0.7,
                  x=-0.15,
                  xanchor='left',
                  yanchor='bottom',
                  buttons=[dict(label='Play',
                                 method='animate',
                                 args=[None, dict(frame=dict(duration=5, redraw=True), 
                                                             transition=dict(duration=4),
                                                             fromcurrent=True,
                                                            #  mode='immediate'
                                                            )]
                                            )
                                      ]
                              )
                        ])

# fig8.show()
# plotly.offline.plot(fig8, filename=r'C:\Users\Giammarco\Desktop\PYTHON_graphs\OUTPUTS\test_single.html')

In [ ]:
# recall variables from 3D_MFPDAs_theory
%store -r X
%store -r Y
%store -r Z
xth=X.reshape(-1)
yth=Y.reshape(-1)
zth=Z.reshape(-1)
d_matrix = np.sqrt(X**2+Y**2+Z**2)
dth=d_matrix.reshape(-1)

In [ ]:
def rot3d(alpha,beta,gamma):  #planar rotation of alpha radians
    return np.array([[np.cos(alpha)*np.cos(beta), np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)], 
                     [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
                     [-np.sin(beta),              np.cos(beta)*np.sin(gamma),                                           np.cos(beta)*np.cos(gamma)]])

frames=[]

camera = dict(
    up=dict(x=0, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=0)
)


fig9 = go.Figure()

# fig9.update_scenes(camera=camera)


fig9 = make_subplots(rows=1, cols=2,
                     subplot_titles=('Experiment', 'Theory'),
                     specs=[[{"type": "mesh3d"}, {"type": "surface"}]],)


fig9.add_trace(go.Mesh3d(x=xgo, y=ygo, z=zgo,i=tigo, j=tjgo, k=tkgo, intensity=dgo,
                        coloraxis="coloraxis1",
                        #    colorscale="Viridis",
                     #       colorbar_len=0.75,
                     #       flatshading=True,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05,facenormalsepsilon=0,vertexnormalsepsilon=0),lightposition=dict(x=100,y=100,z=1000)),1,1)
        
fig9.add_trace(go.Surface(x=X, y=Y, z=Z,
                          surfacecolor=d_matrix,
                          connectgaps=True,
                        coloraxis="coloraxis2",

                        #   colorscale="Viridis",
                     #       colorbar_len=0.75,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05),lightposition=dict(x=100,y=100,z=1000)),1,2)



T = np.arange(0,  2*np.pi, 0.125)
xyzm = np.stack((xgo, ygo, zgo))
xyzs = np.stack((X.reshape(-1), Y.reshape(-1), Z.reshape(-1)))
for alpha in T:
    xrm, yrm, zrm  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyzm)#  batch rotation of all points (x,y,z) on the Mesh
    xrs, yrs, zrs  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyzs)#  batch rotation of all points (x,y,z) on the Mesh
    dths= np.sqrt(xrs**2+xrs**2+zrs**2)
    frames.append(go.Frame(data=[go.Mesh3d(x=xrm, y=yrm, z=zrm),
                                 go.Surface(x=xrs.reshape(200,100), 
                                            y=yrs.reshape(200,100),
                                            z=zrs.reshape(200,100),
                                            surfacecolor=d_matrix)],
                           traces=[0,1])) 
fig9.update(frames=frames)

fig9.update_layout(scene=dict(xaxis_range=[-28,28], xaxis_visible=False, xaxis_autorange=False,
                              yaxis_range=[-28,28], yaxis_visible=False, yaxis_autorange=False,
                              zaxis_range=[-28,28], zaxis_visible=False, zaxis_autorange=False),
                  scene2=dict(xaxis_range=[-0.006, 0.006], xaxis_visible=False, xaxis_autorange=False,
                              yaxis_range=[-0.006, 0.006], yaxis_visible=False, yaxis_autorange=False,
                              zaxis_range=[-0.006, 0.006], zaxis_visible=False, zaxis_autorange=False),
                  coloraxis1=dict(colorscale='Viridis', colorbar=dict(x=.05, xanchor='left', thickness=20)),
                  coloraxis2=dict(colorscale='Viridis', colorbar=dict(x=1.05, xanchor='right', thickness=20)),

                  margin=dict(l=, r=20, t=20, b=20),

                  # scene2=dict(xaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False),
                  #             yaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False),
                  #             zaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False)),
                                             
                  updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=0.7,
                  x=-0.15,
                  xanchor='left',
                  yanchor='bottom',
                  buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=0, redraw=True), 
                                                                     transition=dict(duration=0,easing="quadratic-in-out"),
                                                                     fromcurrent=True,
                                                                  #    mode='immediate'
                                                                  )]),
                           dict(label='Pause', method='animate', args=[None, dict(frame=dict(duration=0, redraw=False), 
                                                                      mode='immediate')])


                          ]
                      )]
                  )

# fig9.show()
plotly.offline.plot(fig9, filename=r'C:\Users\Giammarco\Desktop\PYTHON_graphs\OUTPUTS\test_double.html')

In [ ]:
print(fig9.layout)